In [1]:
first_run = True

if 'google.colab' in str(get_ipython()):
  print("Running in colab...")
  
  !pip install h2o

  from google.colab import drive
  drive.mount('/content/drive')

  train_all_features = '/content/drive/MyDrive/Repasse Gui/dataset_balanced/mean question/df_auto_balanced_labeled.csv'
  validation_dataset_path = '/content/drive/MyDrive/Repasse Gui/dataset_anonymized/all_features_validation.csv'
  path_models = "/content/drive/MyDrive/Repasse Gui/models_H2O"
else:
  if first_run:
    !pip install h2o
    !pip install pandas
    !pip install sklearn
    !pip install matplotlib
  
  print("Running locally...")
  
  train_all_features = f"/home/guisales/Desktop/czrm/Repasse Gui/Repasse Gui/dataset_anonymized/all_features_train-test.csv"
  validation_dataset_path = f"/home/guisales/Desktop/czrm/Repasse Gui/Repasse Gui/dataset_anonymized/test_stratified.csv"
  path_models = f"../models_H2O"

Running in colab...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.2 MB 40 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.1-py2.py3-none-any.whl size=177276478 sha256=515ec565d4ccb2533e23e6908c3b81ba3a630302333316dc04f53d393e8895c1
  Stored in directory: /root/.cache/pip/wheels/a1/a1/d9/bb37df368c4635a707e7362d1088450b606041f05aeba5f173
Successfully built h2o
Mounted at /content/drive


In [2]:
import h2o
from h2o.automl import H2OAutoML

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.16" 2022-07-19; OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04); OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6ylx5puh
  JVM stdout: /tmp/tmp6ylx5puh/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6ylx5puh/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_ghzmum
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [9]:
df = h2o.import_file(train_all_features, header=0)
df = df.drop([0])
df = df.drop([0])
df["Y"] = df["Y"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
feat_dict = dict()
idx = 0
for feat in df.columns:
    feat_dict.update({idx:feat})
    idx += 1

feat_dict

{0: 'Atualmente, alguém no seu domicílio tem algum trabalho remunerado?',
 1: 'Se sim, quantos trabalham?',
 2: 'Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)',
 3: 'Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?',
 4: 'cras',
 5: 'creas',
 6: 'A casa onde a família mora é:',
 7: 'Qual é o material predominante na construção das paredes externas no domicílio?',
 8: 'Qual é a forma de abastecimento de água utilizada no domicílio?',
 9: 'O domicílio possui água canalizada em pelo menos um cômodo?',
 10: 'A água utilizada para beber neste domicílio é:',
 11: 'No domicílio, existe banheiro ou sanitário?',
 12: 'O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?',
 13: 'Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?',
 14: 'Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e ad

In [6]:
def get_accuracy(model, test):
    preds = model.predict(test).as_data_frame(use_pandas=True)
    y_hat = preds["predict"]
    y = test["Y"].as_data_frame(use_pandas=True)
    y = y["Y"]

    hits = 0
    for prediction, target in zip(y_hat, y):
        if prediction == target:
            hits += 1

    return hits/len(y)

In [2]:
def recall(tp, fn):
    return tp/(tp+fn)

In [7]:
best_accuracy = 0
best_model = None
best_feature_set = None

In [8]:
response_col = 'Y'

def apply_autoML_for(training):
    aml = H2OAutoML(max_runtime_secs=0, seed=42, nfolds=10)
    display(training)
    aml.train(y=response_col, training_frame=training)

    return aml

## Auto Balanced

In [ ]:
auto_balanced = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

In [ ]:
df_auto_balanced = df[auto_balanced]

In [ ]:
feature_combinations = {
    "auto_balanced" : df[auto_balanced]
}

In [ ]:
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split

for feature_set in feature_combinations:
    df_tmp = feature_combinations[feature_set]
    print(df_tmp)
    train, test = train_test_split(df_tmp.as_data_frame(), test_size=0.1, stratify=df_tmp.as_data_frame()["ia level"])

    train = h2o.H2OFrame(train)
    test = h2o.H2OFrame(test)

    display(Markdown(f"# Summary for {feature_set}"))
    
    auto_ml = apply_autoML_for(train)
    accuracy = get_accuracy(auto_ml.leader, test)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = auto_ml.leader
        best_feature_set = df_tmp
        best_model_name = f"{feature_set}-{best_model.key}-acc:{accuracy}"
    
        print()
        display(Markdown(f"### Best Model Updated: {best_model_name}"))
        print()
    
        saved_to = h2o.save_model(model=best_model, path=path_models, force=True, filename=best_model_name)
        
        print()
        display(Markdown(f"### Best Model Saved To: {saved_to}"))
        print()
    
    print()
    display(Markdown(f"### {feature_set} has accuracy: {accuracy}"))
    print()
    
    display(Markdown(f"### Details of the best model"))
    display(auto_ml.leader)
    try:
        auto_ml.leader.varimp_plot()
    except Exception as e:
        print(e)

  Atualmente, alguém no seu domicílio tem algum trabalho remunerado?    Se sim, quantos trabalham?    Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)    Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?    cras    creas    A casa onde a família mora é:    Qual é o material predominante na construção das paredes externas no domicílio?    Qual é a forma de abastecimento de água utilizada no domicílio?    O domicílio possui água canalizada em pelo menos um cômodo?    A água utilizada para beber neste domicílio é:    No domicílio, existe banheiro ou sanitário?    O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?    Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?    Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?    Existem locais para atividades

# Summary for auto_balanced

"Atualmente, alguém no seu domicílio tem algum trabalho remunerado?","Se sim, quantos trabalham?","Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)",Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?,cras,creas,A casa onde a família mora é:,Qual é o material predominante na construção das paredes externas no domicílio?,Qual é a forma de abastecimento de água utilizada no domicílio?,O domicílio possui água canalizada em pelo menos um cômodo?,A água utilizada para beber neste domicílio é:,"No domicílio, existe banheiro ou sanitário?",O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?,"Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?","Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?","Existem locais para atividades esportivas para crianças, jovens e adolescentes próximo a sua casa? (quadra de futebol, areninha, pista de skate, academia ao ar livre, etc)?",O domicílio está localizado em área de conflito/violência?,ia level
1,0.166667,0.49505,0,0,0,1,1,0,0,0,0,1,0,0,0,1,grave
1,0.166667,0.392865,0,1,0,1,0,0,1,1,1,0,0,0,1,1,sem ia
1,0.166667,0.587077,0,1,0,1,0,0,0,1,0,0,0,0,0,1,leve
1,0.166667,0.296819,1,0,0,0,0,1,1,0,1,1,0,0,0,1,grave
1,0.166667,0.330033,0,0,0,0,0,0,0,1,0,0,0,0,0,1,moderada
0,0,0.314616,0,0,0,1,0,1,1,1,1,1,0,0,0,1,leve
0,0,0.451797,1,0,0,0,0,0,1,1,1,1,0,0,0,1,sem ia
0,0,0.49505,1,0,0,0,0,1,1,1,1,1,1,0,1,1,moderada
0,0,0.216004,0,0,0,0,1,1,1,0,1,1,1,0,0,1,leve
0,0,0.220582,1,0,0,1,1,1,1,1,1,1,0,0,0,1,grave


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%



### Best Model Updated: auto_balanced-StackedEnsemble_AllModels_2_AutoML_1_20221005_172454-acc:0.5667174023338407

### Best Model Saved To: /content/drive/.shortcut-targets-by-id/1v1b2co-vHiLeABDuI7Kl7ht6eRqepmUD/Repasse Gui/models_H2O/auto_balanced-StackedEnsemble_AllModels_2_AutoML_1_20221005_172454-acc:0.5667174023338407

### auto_balanced has accuracy: 0.5667174023338407

### Details of the best model

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_2_AutoML_1_20221005_172454

No summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.28019591489314977
RMSE: 0.5293353520152888
LogLoss: 0.7629226853738198
Null degrees of freedom: 9939
Residual degrees of freedom: 9837
Null deviance: 27559.16858598292
Residual deviance: 15166.902985231538
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  -------------
1377     319     498         288       0.445205  1,105 / 2,482
50       2067    200         209       0.18171   459 / 2,526
362      253     1562        248       0.355876  863 / 2,425
64       226     166         2051      0.181891  456 / 2,507
1853     2865    2426        2796      0.29004   2,883 / 9,940

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.70996
2    0.898189
3    0.976459
4    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.36811506044679027
RMSE: 0.6067248638771863
LogLoss: 1.0145415959760795
Null degrees of freedom: 53214
Residual degrees of freedom: 53123
Null deviance: 147517.89608236338
Residual deviance: 107977.66205973412
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  ---------------
5177     2121    3791        1898      0.601371  7,810 / 12,987
855      9244    1274        2247      0.321292  4,376 / 13,620
2491     1695    7092        1709      0.453915  5,895 / 12,987
927      2429    1620        8645      0.365318  4,976 / 13,621
9450     15489   13777       14499     0.43328   23,057 / 53,215

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.56672
2    0.823321
3    0.945316
4    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
accuracy                 0.566714  0.00653939  0.570267      0.568415      0.568012      0.558174      0.578259      0.556341      0.567173      0.571296      0.561158      0.568044
auc                      nan       0           nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
err                      0.433286  0.00653939  0.429733      0.431585      0.431988      0.441826      0.421741      0.443659      0.432827      0.428704      0.438842      0.431956
err_count                2305.7    41.4113     2272          2271          2312          2362          2258          2386          2310          2318          2289          2279
logloss                  1.01459   0.0124675   1.03227       1.00465       0.998812      1.01664       1.01226       1.03068       1.01359       0.99594       1.02557       1.01546
max_per_class_error      0.601379  0.0124191   0.592

Variable importance plot is not available for this type of model (stackedensemble).


In [ ]:
print(f'Recall grave: {recall(5177,12987)}, \nRecall para leve: {recall(9244, 13620)}, \nRecall para moderada: {recall(7092,12987)}, \nRecall para sem ia: {recall(8645,13621)}')

Recall grave: 0.2850143140277472, 
Recall para leve: 0.4043037088873338, 
Recall para moderada: 0.3532048408785298, 
Recall para sem ia: 0.388260127548729


## Auto Balanced with Feature Selection

In [12]:
auto_balanced_RFE_DTC = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [13]:
df_auto_balanced_RFE_DTC = df[auto_balanced_RFE_DTC]

In [15]:
feature_combinations = {
    "auto_balanced_RFE_DTC" : df[auto_balanced_RFE_DTC]
}

In [16]:
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split

for feature_set in feature_combinations:
    df_tmp = feature_combinations[feature_set]
    print(df_tmp)
    train, test = train_test_split(df_tmp.as_data_frame(), test_size=0.1, stratify=df_tmp.as_data_frame()["Y"])

    train = h2o.H2OFrame(train)
    test = h2o.H2OFrame(test)

    display(Markdown(f"# Summary for {feature_set}"))
    
    auto_ml = apply_autoML_for(train)
    accuracy = get_accuracy(auto_ml.leader, test)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = auto_ml.leader
        best_feature_set = df_tmp
        best_model_name = f"{feature_set}-{best_model.key}-acc:{accuracy}"
    
        print()
        display(Markdown(f"### Best Model Updated: {best_model_name}"))
        print()
    
        saved_to = h2o.save_model(model=best_model, path=path_models, force=True, filename=best_model_name)
        
        print()
        display(Markdown(f"### Best Model Saved To: {saved_to}"))
        print()
    
    print()
    display(Markdown(f"### {feature_set} has accuracy: {accuracy}"))
    print()
    
    display(Markdown(f"### Details of the best model"))
    display(auto_ml.leader)
    try:
        auto_ml.leader.varimp_plot()
    except Exception as e:
        print(e)

  Atualmente, alguém no seu domicílio tem algum trabalho remunerado?    Se sim, quantos trabalham?    Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)    Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?    cras    A casa onde a família mora é:    Qual é o material predominante na construção das paredes externas no domicílio?    Qual é a forma de abastecimento de água utilizada no domicílio?    O domicílio possui água canalizada em pelo menos um cômodo?    A água utilizada para beber neste domicílio é:    No domicílio, existe banheiro ou sanitário?    O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?    Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?    Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?    Existem locais para atividades esportiv

# Summary for auto_balanced_RFE_DTC

"Atualmente, alguém no seu domicílio tem algum trabalho remunerado?","Se sim, quantos trabalham?","Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)",Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?,cras,A casa onde a família mora é:,Qual é o material predominante na construção das paredes externas no domicílio?,Qual é a forma de abastecimento de água utilizada no domicílio?,O domicílio possui água canalizada em pelo menos um cômodo?,A água utilizada para beber neste domicílio é:,"No domicílio, existe banheiro ou sanitário?",O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?,"Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?","Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?","Existem locais para atividades esportivas para crianças, jovens e adolescentes próximo a sua casa? (quadra de futebol, areninha, pista de skate, academia ao ar livre, etc)?",O domicílio está localizado em área de conflito/violência?,Y
0,0,0.412541,0,1,0,0,0,0,1,1,0,0,0,0,1,moderada
0,0,0.412541,1,0,0,0,0,1,1,1,0,0,0,0,1,leve
0,0,0.205745,1,0,1,0,1,1,1,1,1,0,0,0,1,leve
0,0,0.412541,0,1,0,0,0,0,1,0,0,0,0,0,1,grave
0,0,0.412541,1,1,0,0,0,1,1,1,1,1,0,1,1,moderada
0,0,0.330033,1,0,1,0,0,1,1,1,1,0,0,0,1,moderada
0,0,0.330033,1,1,1,0,1,1,1,1,0,0,0,0,1,moderada
0,0,0.412541,0,1,1,0,1,1,1,1,1,1,0,0,1,grave
1,0.166667,0.49277,0,1,1,1,1,0,0,1,1,1,0,1,1,leve
0,0,0.49505,1,1,0,0,1,1,1,1,1,1,0,0,1,moderada


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%



### Best Model Updated: auto_balanced_RFE_DTC-StackedEnsemble_AllModels_2_AutoML_1_20221006_172321-acc:0.59294930038161

### Best Model Saved To: /content/drive/.shortcut-targets-by-id/1v1b2co-vHiLeABDuI7Kl7ht6eRqepmUD/Repasse Gui/models_H2O/auto_balanced_RFE_DTC-StackedEnsemble_AllModels_2_AutoML_1_20221006_172321-acc:0.59294930038161

### auto_balanced_RFE_DTC has accuracy: 0.59294930038161

### Details of the best model

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_2_AutoML_1_20221006_172321

No summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.24983363176452392
RMSE: 0.4998336040769207
LogLoss: 0.6893308034770056
Null degrees of freedom: 9952
Residual degrees of freedom: 9850
Null deviance: 27594.639592979736
Residual deviance: 13721.818974013273
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  -------------
1612     237     394         204       0.341234  835 / 2,447
73       2116    148         163       0.1536    384 / 2,500
443      238     1501        247       0.38205   928 / 2,429
92       182     172         2131      0.173069  446 / 2,577
2220     2773    2215        2745      0.260524  2,593 / 9,953

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.739476
2    0.908771
3    0.977394
4    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.3387892773511076
RMSE: 0.582056077496926
LogLoss: 0.9521998889538779
Null degrees of freedom: 49518
Residual degrees of freedom: 49428
Null deviance: 137302.9941237807
Residual deviance: 94303.97260228402
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  ---------------
5783     1701    3346        1519      0.531703  6,566 / 12,349
729      9136    1012        1618      0.268828  3,359 / 12,495
2705     1561    6368        1659      0.481982  5,925 / 12,293
811      1750    1304        8517      0.312147  3,865 / 12,382
10028    14148   12030       13313     0.39813   19,715 / 49,519

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.60187
2    0.83887
3    0.950241
4    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
accuracy                 0.60186   0.00484947  0.598983      0.599509      0.605866      0.604703      0.593542      0.597286      0.602569      0.609664      0.606272      0.600204
auc                      nan       0           nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
err                      0.39814   0.00484947  0.401017      0.400491      0.394134      0.395297      0.406458      0.402714      0.397431      0.390336      0.393728      0.399796
err_count                1971.5    28.0208     1971          1958          1962          1967          2014          2018          1980          1963          1921          1961
logloss                  0.952174  0.00909811  0.948485      0.949073      0.97218       0.960917      0.94328       0.958126      0.951757      0.943937      0.949082      0.944898
max_per_class_error      0.531579  0.0100103   0.532168 

Variable importance plot is not available for this type of model (stackedensemble).


In [3]:
print(f'Recall grave: {recall(5783,12349)}, \nRecall para leve: {recall(9136, 12495)}, \nRecall para moderada: {recall(6368,12293)}, \nRecall para sem ia: {recall(8517,12382)}')

Recall grave: 0.31893889256562985, 
Recall para leve: 0.4223568027368129, 
Recall para moderada: 0.3412464498151224, 
Recall para sem ia: 0.40753146083544667


## Undersampling with ENN and Oversampling auto

In [ ]:
under_over_auto_balanced = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

In [ ]:
df_under_over_auto_balanced = df[under_over_auto_balanced]

In [ ]:
feature_combinations = {
    "under_over_auto_balanced" : df[under_over_auto_balanced]
}

In [ ]:
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split

for feature_set in feature_combinations:
    df_tmp = feature_combinations[feature_set]
    print(df_tmp)
    train, test = train_test_split(df_tmp.as_data_frame(), test_size=0.1, stratify=df_tmp.as_data_frame()["y"])

    train = h2o.H2OFrame(train)
    test = h2o.H2OFrame(test)

    display(Markdown(f"# Summary for {feature_set}"))
    
    auto_ml = apply_autoML_for(train)
    accuracy = get_accuracy(auto_ml.leader, test)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = auto_ml.leader
        best_feature_set = df_tmp
        best_model_name = f"{feature_set}-{best_model.key}-acc:{accuracy}"
    
        print()
        display(Markdown(f"### Best Model Updated: {best_model_name}"))
        print()
    
        saved_to = h2o.save_model(model=best_model, path=path_models, force=True, filename=best_model_name)
        
        print()
        display(Markdown(f"### Best Model Saved To: {saved_to}"))
        print()
    
    print()
    display(Markdown(f"### {feature_set} has accuracy: {accuracy}"))
    print()
    
    display(Markdown(f"### Details of the best model"))
    display(auto_ml.leader)
    try:
        auto_ml.leader.varimp_plot()
    except Exception as e:
        print(e)

  Atualmente, alguém no seu domicílio tem algum trabalho remunerado?    Se sim, quantos trabalham?    Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)    Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?    cras    creas    A casa onde a família mora é:    Qual é o material predominante na construção das paredes externas no domicílio?    Qual é a forma de abastecimento de água utilizada no domicílio?    O domicílio possui água canalizada em pelo menos um cômodo?    A água utilizada para beber neste domicílio é:    No domicílio, existe banheiro ou sanitário?    O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?    Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?    Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?    Existem locais para atividades

# Summary for under_over_auto_balanced

"Atualmente, alguém no seu domicílio tem algum trabalho remunerado?","Se sim, quantos trabalham?","Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)",Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?,cras,creas,A casa onde a família mora é:,Qual é o material predominante na construção das paredes externas no domicílio?,Qual é a forma de abastecimento de água utilizada no domicílio?,O domicílio possui água canalizada em pelo menos um cômodo?,A água utilizada para beber neste domicílio é:,"No domicílio, existe banheiro ou sanitário?",O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?,"Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?","Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?","Existem locais para atividades esportivas para crianças, jovens e adolescentes próximo a sua casa? (quadra de futebol, areninha, pista de skate, academia ao ar livre, etc)?",O domicílio está localizado em área de conflito/violência?,y
1,0.166667,0.931084,1,0,0,1,0,0,0,0,1,0,0,0,0,0,leve
1,0.166667,1,1,0,0,1,0,1,1,1,1,1,1,0,0,0,grave
0,0,0.212521,1,0,0,0,0,1,1,1,1,1,1,0,0,1,leve
1,0.166667,0.412541,1,0,0,0,0,1,1,1,1,1,0,0,0,0,grave
0,0,0.412541,1,0,0,1,0,1,1,0,1,1,0,1,0,0,grave
0,0,0.412541,0,0,0,0,0,1,1,1,1,1,1,0,0,1,sem ia
1,0.166667,0.475372,0,0,0,1,0,1,1,1,1,1,0,0,1,0,sem ia
0,0,0.412541,0,1,0,1,0,0,1,1,1,0,0,0,0,1,sem ia
1,0.166667,0.536304,1,1,0,1,0,1,1,0,1,1,0,0,1,1,grave
0,0,0.189769,0,1,0,0,0,1,1,1,1,0,0,0,0,1,moderada


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%



### Best Model Updated: under_over_auto_balanced-StackedEnsemble_AllModels_2_AutoML_5_20221005_213701-acc:0.5369352139612719

### Best Model Saved To: /content/drive/.shortcut-targets-by-id/1v1b2co-vHiLeABDuI7Kl7ht6eRqepmUD/Repasse Gui/models_H2O/under_over_auto_balanced-StackedEnsemble_AllModels_2_AutoML_5_20221005_213701-acc:0.5369352139612719

### under_over_auto_balanced has accuracy: 0.5369352139612719

### Details of the best model

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_2_AutoML_5_20221005_213701

No summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.28096473922895093
RMSE: 0.5300610712257136
LogLoss: 0.7626608191887075
Null degrees of freedom: 10053
Residual degrees of freedom: 9940
Null deviance: 27875.604590858173
Residual deviance: 15335.583752246532
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  --------------
1539     200     618         194       0.396707  1,012 / 2,551
91       2004    208         168       0.188992  467 / 2,471
413      183     1693        218       0.324691  814 / 2,507
130      149     261         1985      0.213861  540 / 2,525
2173     2536    2780        2565      0.281778  2,833 / 10,054

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.718222
2    0.896559
3    0.976228
4    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.39185406062263733
RMSE: 0.625982476290381
LogLoss: 1.0640684157681282
Null degrees of freedom: 37644
Residual degrees of freedom: 37537
Null deviance: 104381.1902284521
Residual deviance: 80113.71102318217
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  ---------------
4170     1158    2919        1164      0.556901  5,241 / 9,411
716      5699    1155        1841      0.394432  3,712 / 9,411
2362     1141    4630        1278      0.508023  4,781 / 9,411
853      1744    1377        5438      0.422227  3,974 / 9,412
8101     9742    10081       9721      0.470394  17,708 / 37,645

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.529605
2    0.800053
3    0.930588
4    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
accuracy                 0.529568  0.00795948  0.528414      0.529587      0.519027      0.527038      0.529805      0.527596      0.537549      0.547396      0.527561      0.521704
auc                      nan       0           nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
err                      0.470432  0.00795948  0.471586      0.470413      0.480973      0.472962      0.470195      0.472404      0.462451      0.452604      0.472439      0.478296
err_count                1770.8    30.3014     1751          1733          1820          1793          1759          1806          1755          1738          1757          1796
logloss                  1.06418   0.0122768   1.06421       1.08415       1.07205       1.05611       1.05839       1.05673       1.05496       1.04455       1.07193       1.07872
max_per_class_error      0.556934  0.0108721   0.5637

Variable importance plot is not available for this type of model (stackedensemble).


In [ ]:
print(f'Recall grave: {recall(4170,9411)}, \nRecall para leve: {recall(5699, 9411)}, \nRecall para moderada: {recall(4630,9411)}, \nRecall para sem ia: {recall(5438,9412)}')

Recall grave: 0.307046609233488, 
Recall para leve: 0.3771674387822634, 
Recall para moderada: 0.32974859340502816, 
Recall para sem ia: 0.3661952861952862


## Undersampling with ENN and Oversampling Pass by Pass

In [ ]:
under_over_passbypass_balanced = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

In [ ]:
df_under_over_passbypass_balanced = df[under_over_passbypass_balanced]

In [ ]:
feature_combinations = {
    "under_over_passbypass_balanced" : df[under_over_passbypass_balanced]
}

In [ ]:
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split

for feature_set in feature_combinations:
    df_tmp = feature_combinations[feature_set]
    print(df_tmp)
    train, test = train_test_split(df_tmp.as_data_frame(), test_size=0.1, stratify=df_tmp.as_data_frame()["y"])

    train = h2o.H2OFrame(train)
    test = h2o.H2OFrame(test)

    display(Markdown(f"# Summary for {feature_set}"))
    
    auto_ml = apply_autoML_for(train)
    accuracy = get_accuracy(auto_ml.leader, test)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = auto_ml.leader
        best_feature_set = df_tmp
        best_model_name = f"{feature_set}-{best_model.key}-acc:{accuracy}"
    
        print()
        display(Markdown(f"### Best Model Updated: {best_model_name}"))
        print()
    
        saved_to = h2o.save_model(model=best_model, path=path_models, force=True, filename=best_model_name)
        
        print()
        display(Markdown(f"### Best Model Saved To: {saved_to}"))
        print()
    
    print()
    display(Markdown(f"### {feature_set} has accuracy: {accuracy}"))
    print()
    
    display(Markdown(f"### Details of the best model"))
    display(auto_ml.leader)
    try:
        auto_ml.leader.varimp_plot()
    except Exception as e:
        print(e)

  Atualmente, alguém no seu domicílio tem algum trabalho remunerado?    Se sim, quantos trabalham?    Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)    Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?    cras    creas    A casa onde a família mora é:    Qual é o material predominante na construção das paredes externas no domicílio?    Qual é a forma de abastecimento de água utilizada no domicílio?    O domicílio possui água canalizada em pelo menos um cômodo?    A água utilizada para beber neste domicílio é:    No domicílio, existe banheiro ou sanitário?    O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?    Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?    Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?    Existem locais para atividades

# Summary for under_over_passbypass_balanced

"Atualmente, alguém no seu domicílio tem algum trabalho remunerado?","Se sim, quantos trabalham?","Ao todo, quanto as pessoas da família ganharam no mês passado (incluindo pensão, bolsa família, cartão mais infância)",Você ou alguém no seu domicílio gostaria de fazer algum curso de qualificação?,cras,creas,A casa onde a família mora é:,Qual é o material predominante na construção das paredes externas no domicílio?,Qual é a forma de abastecimento de água utilizada no domicílio?,O domicílio possui água canalizada em pelo menos um cômodo?,A água utilizada para beber neste domicílio é:,"No domicílio, existe banheiro ou sanitário?",O carro de lixo da Prefeitura busca o seu lixo quantas vezes por semana?,"Existem lugares públicos (praças, parques, etc) para as crianças brincarem próximo a sua casa?","Existem atividades culturais (shows, teatro, cinema, bale, dança, etc) para crianças, jovens e adolescentes próximo a sua casa?","Existem locais para atividades esportivas para crianças, jovens e adolescentes próximo a sua casa? (quadra de futebol, areninha, pista de skate, academia ao ar livre, etc)?",O domicílio está localizado em área de conflito/violência?,y
0,0,0.412541,0,0,0,0,0,0,0,1,1,1,0,0,0,1,moderada
0,0,0.330033,1,0,0,1,0,1,1,1,1,1,1,0,1,1,moderada
1,0.166667,0.660066,1,0,0,1,0,0,1,1,1,1,0,0,0,1,sem ia
0,0,0.299505,1,0,0,1,1,0,1,1,1,0,0,0,1,1,grave
0,0,0.412541,1,1,0,1,0,0,0,1,0,0,0,0,0,1,grave
0,0,0.276403,0,1,0,0,0,1,1,1,1,1,0,0,1,1,grave
0,0,0.584981,1,0,0,1,0,0,1,1,1,1,0,0,0,0,sem ia
0,0,0.25495,0,0,0,1,0,1,1,1,1,1,0,0,0,0,grave
0,0,0.286304,1,0,0,0,0,1,0,1,1,1,0,0,0,1,grave
0,0,0.462046,1,0,0,0,0,1,1,1,1,1,0,0,0,1,moderada


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%



### Best Model Updated: under_over_passbypass_balanced-StackedEnsemble_AllModels_1_AutoML_1_20221006_04938-acc:0.5472149175233086

### Best Model Saved To: /content/drive/.shortcut-targets-by-id/1v1b2co-vHiLeABDuI7Kl7ht6eRqepmUD/Repasse Gui/models_H2O/under_over_passbypass_balanced-StackedEnsemble_AllModels_1_AutoML_1_20221006_04938-acc:0.5472149175233086

### under_over_passbypass_balanced has accuracy: 0.5472149175233086

### Details of the best model

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_1_20221006_04938

No summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.2821949899753141
RMSE: 0.5312202838515432
LogLoss: 0.7665582444304357
Null degrees of freedom: 10051
Residual degrees of freedom: 9960
Null deviance: 27870.06015718034
Residual deviance: 15410.886946029479
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  --------------
1441     220     570         220       0.412077  1,010 / 2,451
93       2134    199         144       0.16965   436 / 2,570
412      210     1665        223       0.336653  845 / 2,510
121      190     243         1967      0.219754  554 / 2,521
2067     2754    2677        2554      0.283028  2,845 / 10,052

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.716972
2    0.897334
3    0.973538
4    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.39392631166891773
RMSE: 0.6276354926778104
LogLoss: 1.0702666948882094
Null degrees of freedom: 37644
Residual degrees of freedom: 37553
Null deviance: 104379.17593126954
Residual deviance: 80580.37945813325
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
grave    leve    moderada    sem ia    Error     Rate
-------  ------  ----------  --------  --------  ---------------
4172     1171    2922        1146      0.556689  5,239 / 9,411
717      5736    1164        1794      0.3905    3,675 / 9,411
2388     1149    4586        1288      0.512698  4,825 / 9,411
813      1843    1423        5333      0.433383  4,079 / 9,412
8090     9899    10095       9561      0.473317  17,818 / 37,645

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.526683
2    0.79923
3    0.93027
4    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
accuracy                 0.526703  0.00707154  0.518837      0.528773      0.523784      0.52651       0.525134      0.533089      0.53321       0.513154      0.536972      0.527563
auc                      nan       0           nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
err                      0.473297  0.00707154  0.481163      0.471227      0.476216      0.47349       0.474866      0.466911      0.46679       0.486846      0.463028      0.472437
err_count                1781.8    39.6086     1788          1736          1802          1795          1776          1785          1771          1869          1722          1774
logloss                  1.07023   0.0106626   1.07986       1.06544       1.07637       1.0732        1.06695       1.05793       1.06341       1.08824       1.05357       1.07735
max_per_class_error      0.556493  0.019076    0.561966 

Variable importance plot is not available for this type of model (stackedensemble).


In [ ]:
print(f'Recall grave: {recall(4172,9411)}, \nRecall para leve: {recall(5763, 9438)}, \nRecall para moderada: {recall(4586,9411)}, \nRecall para sem ia: {recall(5333,9412)}')

Recall grave: 0.3071486416844585, 
Recall para leve: 0.37911979475034535, 
Recall para moderada: 0.32764163749374864, 
Recall para sem ia: 0.36168192607663613
